In [69]:
size=1000
df = pd.DataFrame({'c1': np.random.normal(50,7,size)})
df

,c1
0,50.544144
1,52.357441
2,52.994211
3,51.669300
4,48.857733
...,...
995,48.034286
996,39.046614
997,42.190209
998,37.756242


In [73]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def estimate_sample_size(df, metric_name, effects, alpha=0.05, beta=0.2):
    """Оцениваем sample size для списка эффектов.

    df - pd.DataFrame, датафрейм с данными
    metric_name - str, название столбца с целевой метрикой
    effects - List[float], список ожидаемых эффектов. Например, [1.03] - увеличение на 3%
    alpha - float, ошибка первого рода
    beta - float, ошибка второго рода

    return - pd.DataFrame со столбцами ['effect', 'sample_size']    
    """
    # YOUR_CODE_HERE
    mu = df[metric_name].mean()
    std = df[metric_name].std(ddof=0)
    
    t_alpha = norm.ppf(1-alpha/2, loc=0, scale=1)
    t_beta = norm.ppf(1-beta, loc=0, scale=1)
    t_a_b = (t_alpha + t_beta) ** 2
    num = t_a_b * 2 * (std ** 2)
    
    sample_sizes = []
    for effect in effects:
        eps = mu * (effect - 1)
        sample_sizes.append(num / (eps ** 2))
    return pd.DataFrame({'effect': effects, 'sample_size': np.ceil(sample_sizes).astype(int)})

In [74]:
sample_sizes = estimate_sample_size(df, 'c1', [1.03, 1.1])
sample_sizes

,effect,sample_size
0,1.03,1
1,1.10,1


In [72]:
metric_name = 'c1'
mu = df[metric_name].mean()
std = df[metric_name].std()
alpha = 0.05


for row in sample_sizes.iterrows():
    effect, sample_size = row[1]
    sample_size = int(sample_size)
    result_aa = []
    result_ab = []
    for _ in range(10000):
        a_one = np.random.normal(mu, std, sample_size)
        a_two = np.random.normal(mu, std, sample_size)
        b = np.random.normal(mu*effect, std, sample_size)
        
        _, p_val = ttest_ind(a_one, a_two)
        result_aa.append(int(p_val < alpha))
        
        _, p_val = ttest_ind(a_one, b)
        result_ab.append(int(p_val < alpha))
        
    error_first = np.mean(result_aa)
    error_second = 1 - np.mean(result_ab)
    print(f'effect: {effect}; calculated sample_size: {sample_size}')
    print(f'error_first: {error_first}; error_second: {error_second}\n')

effect: 1.03; calculated sample_size: 327
error_first: 0.0528; error_second: 0.19920000000000004

effect: 1.1; calculated sample_size: 30
error_first: 0.0508; error_second: 0.19940000000000002

